In [1]:
import numpy as np
import torch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import logging
from time import time

# 1 Image per Class

In [2]:
input = torch.load('<filepath>')        #<------Use filepath to the 1ipc .pt file

#Check visualization of a single synthetic set without labels
# print(input['data'][0][0])

#Reshape tensor objects from batch_size, channels, height, width -> batch_size, height, width, channels using torch.permute() and numpy
#Create list of tensor objects without labels
#Framework conflict: Must convert PyTorch tensors into TensorFlow tensors by converting list into numpy array then converting to TF tensors
X=[]
for i in range(5):
    permuted = input['data'][i][0].permute(0, 2, 3, 1)
    numpy_X = permuted.numpy()
    tf_X = tf.convert_to_tensor(numpy_X, dtype = tf.float32)
    X.append(tf_X)

#Split image data into 60, 20, 20 for training, testing, and cross-validation sets
X_train = X[:3]
X_cv = X[3]
X_test = X[4]

#Extract input_shape of a single image: (height, width, channels)
batch_size, height, width, channels = X_train[0].shape
print('Batch_size: ', batch_size, ' Height: ', height, ' Width: ', width, ' Channels: ', channels)

#Create list of label tensor objects
#Convert PyTorch tensors to TensorFlow tensors
y=[]
for i in range(5):
    numpy_y = input['data'][i][1].numpy()
    tf_y = tf.convert_to_tensor(numpy_y, dtype = tf.float32)
    y.append(tf_y)

#Split labels into 60, 20, 20 for training, testing, and cross-validation sets
y_train = y[:3]
y_cv = y[3]
y_test = y[4]

Batch_size:  10  Height:  28  Width:  28  Channels:  1


In [3]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)

#Set global seed so that results don't vary across runs
tf.random.set_seed(1234)

model = Sequential(
    [
    #conv layer 1 (relu)
    Conv2D(input_shape = (height, width, channels), filters=6, kernel_size=5, strides=1, padding='same', activation='relu', name='conv1'),
    #avg pooling
    AveragePooling2D(pool_size=2, strides=2, name='pooling1'),
    #conv layer 2 (relu)
    Conv2D(filters=16, kernel_size=5, strides=1, padding='valid', activation='relu', name='conv2'),
    #avg pooling
    AveragePooling2D(pool_size=2, strides=2, name='pooling2'),
    #conv layer 3 (relu)
    Conv2D(filters=120, kernel_size=5, strides=1, padding='valid', activation='relu', name='conv3'),
    #flatten
    Flatten(),
    #fully connected layer 1 (relu)
    Dense(84, activation='relu', name='dense1', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
    #fully connected layer 2 (linear)
    Dense(10, activation='linear', name='dense2')
    ], name='LeNet-5'
)
# filters are the same as output channel

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-2),
    metrics=['accuracy']
)

In [4]:
start = time()

#Initialize lists so that their values are reset every time the Jupyter notebook is run
X_train_combined=[]
y_train_combined=[]
X_temp=X_train[0]
y_temp=y_train[0]

#Essentially a running sum where you add the ith + 1 element in each iteration until reaching the end of the list
for i in range(len(X_train)):
    try:
        X_temp = np.concatenate((X_temp, X_train[i+1]), axis=0)
        y_temp = np.concatenate((y_temp, y_train[i+1]), axis=0)
    except:
        X_train_combined = X_temp
        y_train_combined = y_temp
        break

#Display duration
duration = time() - start
print(f'Took {duration:.5f} seconds')

model.fit(X_train_combined, y_train_combined, epochs=300, validation_data=(X_cv, y_cv))

Took 0.00067 seconds
Epoch 1/300
1/1 [==============================] - 0s 297ms/step - loss: 12.2589 - accuracy: 0.0000e+00 - val_loss: 10.4698 - val_accuracy: 0.1000
Epoch 2/300
1/1 [==============================] - 0s 18ms/step - loss: 10.4714 - accuracy: 0.1000 - val_loss: 8.6898 - val_accuracy: 0.5000
Epoch 3/300
1/1 [==============================] - 0s 17ms/step - loss: 8.6919 - accuracy: 0.5000 - val_loss: 7.0614 - val_accuracy: 0.6000
Epoch 4/300
1/1 [==============================] - 0s 19ms/step - loss: 7.0609 - accuracy: 0.6000 - val_loss: 5.5503 - val_accuracy: 0.7000
Epoch 5/300
1/1 [==============================] - 0s 18ms/step - loss: 5.5540 - accuracy: 0.7333 - val_loss: 4.5333 - val_accuracy: 0.9000
Epoch 6/300
1/1 [==============================] - 0s 20ms/step - loss: 4.5296 - accuracy: 0.9000 - val_loss: 3.8061 - val_accuracy: 0.9000
Epoch 7/300
1/1 [==============================] - 0s 20ms/step - loss: 3.8037 - accuracy: 0.9000 - val_loss: 2.9204 - val_accuracy

In [5]:
model.summary()

Model: "LeNet-5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 28, 28, 6)         156       
                                                                 
 pooling1 (AveragePooling2D  (None, 14, 14, 6)         0         
 )                                                               
                                                                 
 conv2 (Conv2D)              (None, 10, 10, 16)        2416      
                                                                 
 pooling2 (AveragePooling2D  (None, 5, 5, 16)          0         
 )                                                               
                                                                 
 conv3 (Conv2D)              (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0   

In [6]:
# Calculate the categorization error
#y: target value
#yhat: predicted value
#cerr: % incorrect

def eval_cat_err(y, yhat):
    m = len(y)
    incorrect = 0
    for i in range(m):
       if yhat[i] != y[i]:
            incorrect+=1
    cerr = incorrect / m
    
    return(cerr)

In [7]:
#make a model for plotting routines to call
model_predict = lambda Xl: np.argmax(tf.nn.softmax(model.predict(Xl)).numpy(),axis=1)

training_cerr = eval_cat_err(y_train_combined, model_predict(X_train_combined))
cv_cerr = eval_cat_err(y_cv, model_predict(X_cv))
test_cerr = eval_cat_err(y_test, model_predict(X_test))
print(f"categorization error, training, regularized, 1ipc: {training_cerr:0.7f}" )
print(f"categorization error, cv,       regularized, 1ipc: {cv_cerr:0.7f}" )
print(f"categorization error, test,     regularized, 1ipc: {test_cerr:0.7f}" )

1/1 [==============================] - 0s 10ms/step
categorization error, training, regularized, 1ipc: 0.0000000
categorization error, cv,       regularized, 1ipc: 0.0000000
categorization error, test,     regularized, 1ipc: 0.0000000


# 10 Images per Class

In [8]:
input_10 = torch.load('<filepath>')      #<------Use filepath to the 10ipc .pt file

# print(input_10['data'][0])

#Create list of tensor objects without labels
X_10=[]
for i in range(5):
    permuted = input_10['data'][i][0].permute(0, 2, 3, 1)
    numpy_X = permuted.numpy()
    tf_X = tf.convert_to_tensor(numpy_X, dtype = tf.float32)
    X_10.append(tf_X)

#Split image data into 60, 20, 20 for training, testing, and cross-validation sets
X10_train = X_10[:3]
X10_cv = X_10[3]
X10_test = X_10[4]

batch_size_10, height_10, width_10, channels_10 = X10_train[0].shape
print('Batch_size: ', batch_size_10, ' Height: ', height_10, ' Width: ', width_10, ' Channels: ', channels_10)

#Create list of label tensor objects
y_10=[]
for i in range(5):
    numpy_y = input_10['data'][i][1].numpy()
    tf_y = tf.convert_to_tensor(numpy_y, dtype = tf.float32)
    y_10.append(tf_y)

#Split labels into 60, 20, 20 for training, testing, and cross-validation sets
y10_train = y_10[:3]
y10_cv = y_10[3]
y10_test = y_10[4]

Batch_size:  100  Height:  28  Width:  28  Channels:  1


In [9]:
#To prevent an excessive messages when running
logging.getLogger("tensorflow").setLevel(logging.ERROR)

#Set global seed so that results don't vary across runs
tf.random.set_seed(1234)

model_10 = Sequential(
    [
    #conv layer 1 (relu)
    Conv2D(input_shape = (height_10, width_10, channels_10), filters=6, kernel_size=5, strides=1, padding='same', activation='relu', name='conv1'),
    #avg pooling
    AveragePooling2D(pool_size=2, strides=2, name='pooling1'),
    #conv layer 2 (relu)
    Conv2D(filters=16, kernel_size=5, strides=1, padding='valid', activation='relu', name='conv2'),
    #avg pooling
    AveragePooling2D(pool_size=2, strides=2, name='pooling2'),
    #conv layer 3 (relu)
    Conv2D(filters=120, kernel_size=5, strides=1, padding='valid', activation='relu', name='conv3'),
    #flatten
    Flatten(),
    #fully connected layer 1 (relu)
    Dense(84, activation='relu', name='dense1', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
    #fully connected layer 2 (linear)
    Dense(10, activation='linear', name='dense2')
    ], name='LeNet-5'
)
# filters are the same as output channel

model_10.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-2),
    metrics=['accuracy']
)

In [10]:
#With 10ipc (images per class), we have batch sizes of 100 so the train set size is 300

start = time()

#Initialize lists so that their values are reset every time the Jupyter notebook is run
X10_train_combined=[]
y10_train_combined=[]
X_temp=X10_train[0]
y_temp=y10_train[0]

#Essentially a running sum where you add the ith + 1 element in each iteration until reaching the end of the list
for i in range(len(X10_train)):
    try:
        X_temp = np.concatenate((X_temp, X10_train[i+1]), axis=0)
        y_temp = np.concatenate((y_temp, y10_train[i+1]), axis=0)
    except:
        X10_train_combined = X_temp
        y10_train_combined = y_temp
        break

# calculate and report duration of concatenation
duration = time() - start
print(f'Took {duration:.5f} seconds')

model_10.fit(X10_train_combined, y10_train_combined, epochs=300, validation_data=(X10_cv, y10_cv))

Took 0.00087 seconds
Epoch 1/300
10/10 [==============================] - 0s 15ms/step - loss: 7.1843 - accuracy: 0.0867 - val_loss: 3.1149 - val_accuracy: 0.3400
Epoch 2/300
10/10 [==============================] - 0s 6ms/step - loss: 2.4624 - accuracy: 0.2833 - val_loss: 2.1591 - val_accuracy: 0.3900
Epoch 3/300
10/10 [==============================] - 0s 6ms/step - loss: 1.8507 - accuracy: 0.4833 - val_loss: 1.4870 - val_accuracy: 0.5900
Epoch 4/300
10/10 [==============================] - 0s 6ms/step - loss: 1.3246 - accuracy: 0.6633 - val_loss: 1.1167 - val_accuracy: 0.7100
Epoch 5/300
10/10 [==============================] - 0s 6ms/step - loss: 1.0184 - accuracy: 0.7600 - val_loss: 0.9468 - val_accuracy: 0.7300
Epoch 6/300
10/10 [==============================] - 0s 7ms/step - loss: 0.8926 - accuracy: 0.7967 - val_loss: 0.7917 - val_accuracy: 0.8900
Epoch 7/300
10/10 [==============================] - 0s 5ms/step - loss: 0.5646 - accuracy: 0.9200 - val_loss: 0.5027 - val_accuracy

In [11]:
model_10.summary()

Model: "LeNet-5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 28, 28, 6)         156       
                                                                 
 pooling1 (AveragePooling2D  (None, 14, 14, 6)         0         
 )                                                               
                                                                 
 conv2 (Conv2D)              (None, 10, 10, 16)        2416      
                                                                 
 pooling2 (AveragePooling2D  (None, 5, 5, 16)          0         
 )                                                               
                                                                 
 conv3 (Conv2D)              (None, 1, 1, 120)         48120     
                                                                 
 flatten_1 (Flatten)         (None, 120)               0   

In [12]:
#make a model for plotting routines to call
model_predict_10 = lambda Xl: np.argmax(tf.nn.softmax(model_10.predict(Xl)).numpy(),axis=1)

training_cerr_10 = eval_cat_err(y10_train_combined, model_predict_10(X10_train_combined))
cv_cerr_10 = eval_cat_err(y10_cv, model_predict_10(X10_cv))
test_cerr_10 = eval_cat_err(y10_test, model_predict_10(X10_test))
print(f"categorization error, training, regularized, 10ipc: {training_cerr_10:0.5f}" )
print(f"categorization error, cv,       regularized, 10ipc: {cv_cerr_10:0.5f}" )
print(f"categorization error, test,     regularized, 10ipc: {test_cerr_10:0.5f}" )

4/4 [==============================] - 0s 1ms/step
categorization error, training, regularized, 10ipc: 0.00000
categorization error, cv,       regularized, 10ipc: 0.01000
categorization error, test,     regularized, 10ipc: 0.01000
